# Crear Registros

Los objetivos de aprendizaje son:

1. Datos en Código vs Datos en Bases de Datos
2. Crear Datos con SQLModel
    - Instancias de modelos de datos
    - Crear una session
    - Añadir instancias a la sesión
    - Commit
    - Cerrar la sesión.
    

## Datos en Código vs Datos en Bases de Datos

Al trabajar con una base de datos en un lenguaje de programación existirán datos en: 

- **La memoria**: Almacenamiento temporal, e.g. objetos y variables que creamos en nuestro código.
<br>

- **Base de datos**: Almacenamiento permanente gestionados por una aplicación externa.

Esto implica que:

1. Obtendremos datos de la BD y los almacenaremos en la memoria dentro de variables.
<br>

2. Crearemos objetos que querremos guardar en la BD, y que enviaremos de alguna manera.


## Crear Datos con SQLMode

Dado que Python y la Base de Datos son sistemas independientes deberemos seguir los siguientes pasos:

1. Crear los datos en Python, e.g. en una variable.
<br>

2. guardar/enviar los datos a la base de datos.

### Instancias de modelos de datos

Ya creamos una clase `Asegurado` que representa la tabla de `asegurado` en la base de datos.

Cada instancia de la clase `Asegurado` que creemos representará una fila en la tabla `asegurado`.

Entonces, el primer paso es simplemente crear una instancia de clase `Asegurado`.

Crearemos 4:

In [1]:
from crear_base_de_datos import Asegurado

asegurado_1 = Asegurado(nombre="Heber", lob="Autos", edad=30)
asegurado_2 = Asegurado(nombre="Joaquim", lob="Hogar", edad=35)
asegurado_3 = Asegurado(nombre="Antonio", lob="Autos", edad=40)
asegurado_4 = Asegurado(nombre="Juan", lob="Hogar", edad=25)
    
asegurado_1

2024-02-05 19:05:46,675 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-05 19:05:46,675 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("asegurado")
2024-02-05 19:05:46,675 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-05 19:05:46,676 INFO sqlalchemy.engine.Engine COMMIT


Asegurado(id=None, nombre='Heber', lob='Autos', edad=30)

>**Nota**: El atributo `id` es `None`, pero al mismo tiempo la BD exige que sea único y diferente de `NULL`. Esto es así porque la BD se encargará de gestionar la llave primaria.

### Crear una session

Hasta ahora solo hemos utilizado la clase `Engine` para interactuar con la base de datos.

El `engine` debe ser un único objeto que comparta toda nuestra aplicación, y que se encarga de comunicarse con la base de datos, gestionar las conexiones, etc.

En la práctica no es recomendable usar directamente un `Engine`, es mejor utilizar otra herramienta que se construye sobre el `Engine`, la clase `Session`.

A diferencia del `engine` que es único para toda la aplicación, podemos crear una instancia nueva de la clase `Session` para cada grupo de operaciones que querramos enviar a la BD.

Es decir, cada vez que realizamos una operación(es) crearemos una sesión usando un único `engine`, y una vez finalizada la operación cerraríamos la sesión.

In [2]:
from sqlmodel import Session, create_engine

sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=True)
session = Session(engine)
session

### Añadir instancias a la sesión

Ahora que tenemos algunas instancias del modelo asegurado (algunos objetos en la memoria) y una sesión, el siguiente paso es agregarlos a la sesión:


In [3]:
session.add(asegurado_1)
session.add(asegurado_2)
session.add(asegurado_3)
session.add(asegurado_4)

En este momento, los asegurados `asegurado_<i>` no están almacenados en la base de datos.

La sesión mantiene en memoria todos los objetos que deben guardarse en la base de datos más adelante.

Una vez que estemos listos, podemos confirmar esos cambios, y luego la sesión usará el `engine` para enviar el comando SQL apropiado (`INSERT`) a la base de datos, de esa manera la API de la Base de Datos creará los registros.

Todo en un solo *batch*. Esto hace que las interacciones con la base de datos sean más eficientes.

>**Nota**: Las bases de datos SQL cumplen un conjunto de propiedades conocidas como ACID (atomicity, consistency, isolation, durability). La implementación de `SQLModel` busca cumplir con estas propiedades. 


### Commit

Ahora que tenemos a los asegurados dentro de la sesión, y que estamos listos para guardar todo eso en la base de datos, podemos confirmar los cambios:

In [4]:
session.commit()

2024-02-05 19:10:23,401 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-05 19:10:23,404 INFO sqlalchemy.engine.Engine INSERT INTO asegurado (nombre, lob, edad) VALUES (?, ?, ?)
2024-02-05 19:10:23,405 INFO sqlalchemy.engine.Engine [generated in 0.00093s] ('Heber', 'Autos', 30)
2024-02-05 19:10:23,407 INFO sqlalchemy.engine.Engine INSERT INTO asegurado (nombre, lob, edad) VALUES (?, ?, ?)
2024-02-05 19:10:23,408 INFO sqlalchemy.engine.Engine [cached since 0.00361s ago] ('Joaquim', 'Hogar', 35)
2024-02-05 19:10:23,409 INFO sqlalchemy.engine.Engine INSERT INTO asegurado (nombre, lob, edad) VALUES (?, ?, ?)
2024-02-05 19:10:23,409 INFO sqlalchemy.engine.Engine [cached since 0.004997s ago] ('Antonio', 'Autos', 40)
2024-02-05 19:10:23,410 INFO sqlalchemy.engine.Engine INSERT INTO asegurado (nombre, lob, edad) VALUES (?, ?, ?)
2024-02-05 19:10:23,410 INFO sqlalchemy.engine.Engine [cached since 0.005932s ago] ('Juan', 'Hogar', 25)
2024-02-05 19:10:23,411 INFO sqlalchemy.engine.Engine CO

Después de confirmar los cambios, pasa algo interesante con las instancias de la clase `Asegurado`:

In [5]:
asegurado_1

Asegurado()

Lo que sucede es que `SQLModel` está marcando internamente esos objetos como "caducados", no tienen la última versión de los datos.

Esto se debe a que alguuna otra aplicación podría estar modificando los mismos datos que ya hemos integrado a la BD, e.g. el auto-incremental de la llave primaria.

La próxima vez que accedamos a algún atributo, por ejemplo con:

In [6]:
asegurado_1.nombre

2024-02-05 19:11:27,165 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-05 19:11:27,168 INFO sqlalchemy.engine.Engine SELECT asegurado.id AS asegurado_id, asegurado.nombre AS asegurado_nombre, asegurado.lob AS asegurado_lob, asegurado.edad AS asegurado_edad 
FROM asegurado 
WHERE asegurado.id = ?
2024-02-05 19:11:27,169 INFO sqlalchemy.engine.Engine [generated in 0.00084s] (1,)


'Heber'

`SQLModel` se asegurará de ponerse en contacto con la base de datos y obtener la versión más reciente de los datos:

In [7]:
asegurado_1

Asegurado(edad=30, nombre='Heber', lob='Autos', id=1)

la sesión actualiza los datos automáticamente en segundo plano, y como efecto secundario, cuando accedemos a un atributo.

Pero también podemos hacerlo explícitamente.

In [8]:
asegurado_2

Asegurado()

In [9]:
session.refresh(asegurado_2)
asegurado_2

2024-02-05 19:11:53,762 INFO sqlalchemy.engine.Engine SELECT asegurado.id, asegurado.nombre, asegurado.lob, asegurado.edad 
FROM asegurado 
WHERE asegurado.id = ?
2024-02-05 19:11:53,764 INFO sqlalchemy.engine.Engine [generated in 0.00158s] (2,)


Asegurado(edad=35, nombre='Joaquim', lob='Hogar', id=2)

### Cerrar la sesión

La sesión ocupa algunos recursos, e.g. conexiones del `engine`.

Entonces, una vez que hayamos terminado, debemos cerrar la sesión para que libere esos recursos:

In [10]:
session.close()

2024-02-05 19:12:21,046 INFO sqlalchemy.engine.Engine ROLLBACK


Pero, ¿qué pasa si nos olvidamos de cerrar la sesión?

¿O si hay una excepción en el código y nunca se ejecuta `session.close()`?

Para eso, hay una mejor manera de crear y cerrar la sesión:

```python
with Session(engine) as session:
    session.add(asegurado_1)
    session.add(asegurado_2)
    session.add(asegurado_3)
    session.add(asegurado_4)
    
    session.commit()
```